# Rectangle Algebra

In [28]:
import qualreas as qr
import os
import numpy as np

path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

## Load the 2D Point Algebra

In [2]:
ptalg2d = qr.Algebra(os.path.join(path, "Algebras/2D_Point_Algebra.json"))

In [3]:
ptalg2d.summary()

  Algebra Name: 2D_Point_Algebra
   Description: Autogenerated 2-dimensional point algebra
 Equality Rels: EQ
     Relations:
            NAME (SYMBOL)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE   DOMAIN        RANGE
              South (  S)               North (  N)    False      False       True       2DPt          2DPt
             Equals ( EQ)              Equals ( EQ)     True       True       True       2DPt          2DPt
              North (  N)               South (  S)    False      False       True       2DPt          2DPt
          Southwest ( SW)           Northeast ( NE)    False      False       True       2DPt          2DPt
               West (  W)                East (  E)    False      False       True       2DPt          2DPt
          Northwest ( NW)           Southeast ( SE)    False      False       True       2DPt          2DPt
          Southeast ( SE)           Northwest ( NW)    False      False       True       2DPt          2DPt
           

In [4]:
qr.print_point_algebra_composition_table(ptalg2d)

2D_Point_Algebra
Elements: S, EQ, N, SW, W, NW, SE, E, NE
 rel1 ; rel2 = composition
   S      S      S
   S     EQ      S
   S      N      S|EQ|N
   S     SW      SW
   S      W      SW
   S     NW      SW|W|NW
   S     SE      SE
   S      E      SE
   S     NE      SE|E|NE
------------------------------
  EQ      S      S
  EQ     EQ      EQ
  EQ      N      N
  EQ     SW      SW
  EQ      W      W
  EQ     NW      NW
  EQ     SE      SE
  EQ      E      E
  EQ     NE      NE
------------------------------
   N      S      S|EQ|N
   N     EQ      N
   N      N      N
   N     SW      SW|W|NW
   N      W      NW
   N     NW      NW
   N     SE      SE|E|NE
   N      E      NE
   N     NE      NE
------------------------------
  SW      S      SW
  SW     EQ      SW
  SW      N      SW|W|NW
  SW     SW      SW
  SW      W      SW
  SW     NW      SW|W|NW
  SW     SE      S|SW|SE
  SW      E      S|SW|SE
  SW     NE      S|EQ|N|SW|W|NW|SE|E|NE
------------------------------
   W      S

## Derive Rectangle Relations

TODO: Modify the code below to handle 2DPoints

In [19]:
class Four2DPointNet(qr.Network):
    """Create four 2D Points that represent the coordinates of the upper-right (UR)
    and lower-left (LL) of two rectangles. For example, (LL1, UR1) and (LL2, UR2),
    where LL1 SW UR1, LL2 SW UR2, and SW is the 2DPoint relation southwest,
    represents two proper rectangles. Return the network and the four spatial entities."""
    def __init__(self, algebra, name, southwest="SW", lowerleft="LL", upperright="UR"):
        self.algebra = algebra
        self.SW = algebra.relset(southwest)
        # LL and UR coordinates of rectangle 1
        LL1 = lowerleft + "1"
        UR1 = upperright + "1"
        self.LL1 = qr.SpatialEntity(["2DPoint"], name=LL1)
        self.UR1 = qr.SpatialEntity(["2DPoint"], name=UR1)
        # LL and UR coordinates of rectangle 2
        LL2 = lowerleft + "2"
        UR2 = upperright + "2"
        self.LL2 = qr.SpatialEntity(["2DPoint"], name=LL2)
        self.UR2 = qr.SpatialEntity(["2DPoint"], name=UR2)
        self.name_list = [LL1, UR1, LL2, UR2]
        super().__init__(algebra, name)
        self.add_constraint(self.LL1, self.UR1, self.SW, verbose=False)
        self.add_constraint(self.LL2, self.UR2, self.SW, verbose=False)

    def get_points(self):
        return [self.LL1, self.UR1, self.LL2, self.UR2]

In [20]:
foo = Four2DPointNet(ptalg2d, "Foo")

foo

In [21]:
foo.get_points()

[SpatialEntity(['2DPoint'] 'LL1'),
 SpatialEntity(['2DPoint'] 'UR1'),
 SpatialEntity(['2DPoint'] 'LL2'),
 SpatialEntity(['2DPoint'] 'UR2')]

In [22]:
foo.summary()


Foo: 4 nodes, 8 edges
  Algebra: 2D_Point_Algebra
  LL1:['2DPoint']
    => LL1: EQ
    => UR1: SW
  UR1:['2DPoint']
    => UR1: EQ
  LL2:['2DPoint']
    => LL2: EQ
    => UR2: SW
  UR2:['2DPoint']
    => UR2: EQ


In [23]:
foo.propagate()

True

In [24]:
foo.summary()


Foo: 4 nodes, 16 edges
  Algebra: 2D_Point_Algebra
  LL1:['2DPoint']
    => LL1: EQ
    => UR1: SW
    => LL2: S|EQ|N|SW|W|NW|SE|E|NE
    => UR2: S|EQ|N|SW|W|NW|SE|E|NE
  UR1:['2DPoint']
    => UR1: EQ
    => LL2: S|EQ|N|SW|W|NW|SE|E|NE
    => UR2: S|EQ|N|SW|W|NW|SE|E|NE
  LL2:['2DPoint']
    => LL2: EQ
    => UR2: SW
  UR2:['2DPoint']
    => UR2: EQ


In [29]:
def generate_consistent_2D_networks(point_algebra, lessthan="SW", startname="LL", endname="UR",
                                    verbose=False):
    """For a given point algebra and southwest relation, derive all possible consistent networks
    of 4 points, where the points represent the lower-left and upper-right of 2 rectangles."""
    consistent_nets = dict()
    for elem13 in point_algebra.elements:
        for elem23 in point_algebra.elements:
            for elem14 in point_algebra.elements:
                for elem24 in point_algebra.elements:
                    four_pt_net_name = elem13 + ',' + elem14 + ',' + elem23 + ',' + elem24
                    ptnet = Four2DPointNet(point_algebra, four_pt_net_name, lessthan, startname, endname)
                    pt1, pt2, pt3, pt4 = ptnet.get_points()
                    rs13 = point_algebra.relset(elem13)
                    rs14 = point_algebra.relset(elem14)
                    rs23 = point_algebra.relset(elem23)
                    rs24 = point_algebra.relset(elem24)
                    ptnet.add_constraint(pt1, pt3, rs13)
                    ptnet.add_constraint(pt1, pt4, rs14)
                    ptnet.add_constraint(pt2, pt3, rs23)
                    ptnet.add_constraint(pt2, pt4, rs24)
                    if ptnet.propagate():
                        elem_key = ",".join([str(rs13), str(rs14), str(rs23), str(rs24)])
                        consistent_nets[elem_key] = ptnet
                        if verbose:
                            print(np.array(ptnet.to_list()))
    print(f"\n{len(consistent_nets)} consistent networks")
    return consistent_nets

In [31]:
result = generate_consistent_2D_networks(ptalg2d, verbose=True)

[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'SE' 'S']
 ['N' 'NW' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'SE' 'SW']
 ['N' 'NW' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'SE' 'SE']
 ['N' 'NW' 'EQ' 'SW']
 ['NE' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'E' 'S']
 ['N' 'W' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'E' 'SW']
 ['N' 'W' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'E' 'SE']
 ['N' 'W' 'EQ' 'SW']
 ['NE' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'S']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'EQ']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'EQ' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'N']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'S' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'SW']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'S' 'SW']
 ['NE' 'EQ' 'NE' 'W']
 ['N' 'SW' 'EQ' 'SW']
 ['NE' 'E' 'NE' 'EQ']]
[['EQ' 'SW' 'S'

[['EQ' 'SW' 'E' 'S']
 ['NE' 'EQ' 'NE' 'SE']
 ['W' 'SW' 'EQ' 'SW']
 ['N' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'S']
 ['NE' 'EQ' 'NE' 'E']
 ['W' 'SW' 'EQ' 'SW']
 ['N' 'W' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'S']
 ['NE' 'EQ' 'NE' 'NE']
 ['W' 'SW' 'EQ' 'SW']
 ['N' 'SW' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'SW']
 ['NE' 'EQ' 'NE' 'S']
 ['W' 'SW' 'EQ' 'SW']
 ['NE' 'N' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'SW']
 ['NE' 'EQ' 'NE' 'EQ']
 ['W' 'SW' 'EQ' 'SW']
 ['NE' 'EQ' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'SW']
 ['NE' 'EQ' 'NE' 'N']
 ['W' 'SW' 'EQ' 'SW']
 ['NE' 'S' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'SW']
 ['NE' 'EQ' 'NE' 'SW']
 ['W' 'SW' 'EQ' 'SW']
 ['NE' 'NE' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'SW']
 ['NE' 'EQ' 'NE' 'W']
 ['W' 'SW' 'EQ' 'SW']
 ['NE' 'E' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'SW']
 ['NE' 'EQ' 'NE' 'NW']
 ['W' 'SW' 'EQ' 'SW']
 ['NE' 'SE' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'SW']
 ['NE' 'EQ' 'NE' 'SE']
 ['W' 'SW' 'EQ' 'SW']
 ['NE' 'NW' 'NE' 'EQ']]
[['EQ' 'SW' 'E' 'SW']
 ['NE' 'EQ' 'NE' 'E']
 ['W' 'SW' 'EQ' 'SW']
 ['NE' 'W' 'NE' 'EQ']]
[['EQ' 'SW' 'E'

In [32]:
result

{'S,SW,SE,S': <__main__.Four2DPointNet at 0x7fdb6199fc40>,
 'S,SW,SE,SW': <__main__.Four2DPointNet at 0x7fdb619bf0d0>,
 'S,SW,SE,SE': <__main__.Four2DPointNet at 0x7fdb60dc8580>,
 'S,SW,E,S': <__main__.Four2DPointNet at 0x7fdb52c55520>,
 'S,SW,E,SW': <__main__.Four2DPointNet at 0x7fdb612456d0>,
 'S,SW,E,SE': <__main__.Four2DPointNet at 0x7fdb619869d0>,
 'S,SW,NE,S': <__main__.Four2DPointNet at 0x7fdb619486a0>,
 'S,SW,NE,EQ': <__main__.Four2DPointNet at 0x7fdb61978070>,
 'S,SW,NE,N': <__main__.Four2DPointNet at 0x7fdb52c537f0>,
 'S,SW,NE,SW': <__main__.Four2DPointNet at 0x7fdb6126e190>,
 'S,SW,NE,W': <__main__.Four2DPointNet at 0x7fdb61245640>,
 'S,SW,NE,NW': <__main__.Four2DPointNet at 0x7fdb52bbba30>,
 'S,SW,NE,SE': <__main__.Four2DPointNet at 0x7fdb52bc4130>,
 'S,SW,NE,E': <__main__.Four2DPointNet at 0x7fdb52b7e6d0>,
 'S,SW,NE,NE': <__main__.Four2DPointNet at 0x7fdb52b83df0>,
 'EQ,SW,NE,S': <__main__.Four2DPointNet at 0x7fdb61172130>,
 'EQ,SW,NE,EQ': <__main__.Four2DPointNet at 0x7fd